## 生産計画問題の主問題

In [30]:
from pulp import *
import numpy as np

#　係数行列の定義
A = np.array([[3,1,2],[1,3,0],[0,2,4]])
#　コストベクトルの定義
c = np.array([150,200,300])
#　右ベクトルの定義
b = np.array([60,36,48])

#　m,nに係数行列の列と行の数を格納
(m,n) = A.shape

#　問題の定義
prob = LpProblem(name="Production1",sense=LpMaximize)
#　変数の定義
x = [LpVariable("x"+str(i+1),lowBound=0) for i in range(n)]

#　目的関数の定義
#　lpDot(c,x)でコストベクトルと変数ベクトルの内積
prob += lpDot(c,x)

#　制約式
for i in range(m):
    prob += lpDot(A[i],x) <= b[i], "ineq"+str(i)

#　結果の出力
print(prob)
prob.solve()
print(LpStatus[prob.status])
print("Optimal value =", value(prob.objective))
for v in prob.variables():
    print(v.name,"=",v.varValue)

Production1:
MAXIMIZE
150*x1 + 200*x2 + 300*x3 + 0
SUBJECT TO
ineq0: 3 x1 + x2 + 2 x3 <= 60

ineq1: x1 + 3 x2 <= 36

ineq2: 2 x2 + 4 x3 <= 48

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

Optimal
Optimal value = 5800.0
x1 = 12.0
x2 = 8.0
x3 = 8.0


### 得られた最適解（　主問題(P)　）が実行可能かどうか

In [31]:
X = np.array([v.varValue for v in prob.variables()])
np.all(np.abs(b-np.dot(A,X))<=1.0e-5)

True

## 生産計画問題の双対問題

In [32]:
from pulp import *
import numpy as np

#　係数行列の定義
A = np.array([[3,1,0],[1,3,2],[2,0,4]])
#　右ベクトルの定義
b = np.array([150,200,300])
#　コストベクトルの定義
c = np.array([60,36,48])

#　m,nに係数行列の列と行の数を格納
(m,n) = A.shape

#　問題の定義
prob = LpProblem(name="Production2",sense=LpMinimize)
#　変数の定義
y = [LpVariable("y"+str(i+1),lowBound=0) for i in range(n)]

#　目的関数の定義
#　lpDot(c,x)でコストベクトルと変数ベクトルの内積
prob += lpDot(b,y)

#　制約式
for i in range(m):
    prob += lpDot(A[i],y) >= b[i], "ineq"+str(i)

#　結果の出力
print(prob)
prob.solve()
print(LpStatus[prob.status])
print("Optimal value =", value(prob.objective))
for v in prob.variables():
    print(v.name,"=",v.varValue)

Production2:
MINIMIZE
150*y1 + 200*y2 + 300*y3 + 0
SUBJECT TO
ineq0: 3 y1 + y2 >= 150

ineq1: y1 + 3 y2 + 2 y3 >= 200

ineq2: 2 y1 + 4 y3 >= 300

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous

Optimal
Optimal value = 25833.3334
y1 = 44.444444
y2 = 16.666667
y3 = 52.777778


### 得られた最適解（　双対問題(D））が実行可能かどうか

In [34]:
Y = np.array([v.varValue for v in prob.variables()])
np.all(np.abs(np.dot(A,Y)-b) <= 1.0e-5)

True

### 誤植？？
-cではなく-bならうTrueが返ってくる

In [36]:
Y = np.array([v.varValue for v in prob.variables()])
np.all(np.abs(np.dot(A,Y)-c) <= 1.0e-5)

False

### 誤植ではなくb.cの値を主問題の時と双対問題の時で変えてしまっているから
### 教科書とは全く同じコードでないためにこのようになってしまっている